# Verify GPU
This cell verifies that the GPU is active


> Indented block



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


# Import and install all the needed libraries

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-78evs88p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-78evs88p
  Resolved https://github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8119970 sha256=f9403fabc3a00aed5e45cc942f289b345b46ba97853de9671f978322d86818ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-t3yor0mf/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load Dataset
This cell will load a dataset from mozilla common_voice the language choosen for this project will be Bengali


In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="train[:30%]+validation[:30%]", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "bn", split="test[:30%]", use_auth_token=True)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 7539
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2506
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 7539
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2506
    })
})


In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3176f7d070e0a8030b3742168b5aa12aa22dad56beb5338e7431487ab248be83/bn_train_0/common_voice_bn_31639641.mp3', 'array': array([1.98951966e-13, 1.16529009e-12, 1.44950718e-12, ...,
       6.04368506e-06, 5.44542763e-06, 2.85280657e-06]), 'sampling_rate': 48000}, 'sentence': 'এই ভোগ সম্পূর্ণভাবে নির্ভর করে তার নিজের উপরে।'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3176f7d070e0a8030b3742168b5aa12aa22dad56beb5338e7431487ab248be83/bn_train_0/common_voice_bn_31639641.mp3', 'array': array([6.54836185e-11, 3.27418093e-11, 3.63797881e-11, ...,
       1.87879778e-06, 1.88457489e-06, 5.89941919e-06]), 'sampling_rate': 16000}, 'sentence': 'এই ভোগ সম্পূর্ণভাবে নির্ভর করে তার নিজের উপরে।'}


In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/7539 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2506 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Google drive Issues
A shared folder was setup manually in google drive as one of the accounts had unlimited storage


In [ ]:
common_voice.save_to_disk('/content/drive/Shareddrives/Shared with Redve/New Dataset')

Saving the dataset (0/15 shards):   0%|          | 0/7539 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/2506 [00:00<?, ? examples/s]